In [96]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [112]:
#load the train.csv file into a dataframe
train = pd.read_csv('./petfinder-adoption-prediction/train/train.csv')
#load the test.csv file into a dataframe
test = pd.read_csv('./petfinder-adoption-prediction/test/test.csv')

#drop text/object features from train
train = train.drop(['Name', 'RescuerID', 'Description','PetID'], axis = 1)
#drop text/object features from test
test = test.drop(['Name', 'RescuerID', 'Description','PetID'], axis = 1)

#categorise the features into nominal (categorical) or numerical features
nominal_features = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'State']
numerical_features = ['Age', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'VideoAmt','PhotoAmt']

#instantiate a OneHotEncoder that will output an array
ohe = OneHotEncoder(sparse=False)

#TODO rewrite the nominal feature transformation to handle all nominal features at once
#i.e only have one OneHotEncoder that can then be reused for the test dataset

#for each nominal feature, transform it with the OneHotEncoder, and add it to the dataframe
for nominal in nominal_features:
    train_nominal_encoded = pd.DataFrame (ohe.fit_transform(train[[nominal]]))
    train_nominal_encoded.columns = ohe.get_feature_names_out([nominal])
        
    train.drop([nominal] ,axis=1, inplace=True)
    train = pd.concat([train, train_nominal_encoded ], axis=1)
    
    test_nominal_encoded = pd.DataFrame (ohe.fit_transform(test[[nominal]]))
    test_nominal_encoded.columns = ohe.get_feature_names_out([nominal])
        
    test.drop([nominal] ,axis=1, inplace=True)
    test = pd.concat([test, test_nominal_encoded ], axis=1)
    
#setup the scaler
scaler = StandardScaler()

#scale the numerical features of the train dataframe and replace the unscaled data
train_numerical = train[numerical_features]
train_numerical = scaler.fit_transform(train_numerical)
train_numerical_df = pd.DataFrame(train_numerical, columns=numerical_features)

train = train.drop(numerical_features, axis = 1)
train = pd.concat([train, train_numerical_df], axis=1)

#scale the numerical features of the test dataframe and replace the unscaled data
test_numerical = test[numerical_features]
test_numerical = scaler.fit_transform(test_numerical)
test_numerical_df = pd.DataFrame(test_numerical, columns=numerical_features)

test = test.drop(numerical_features, axis = 1)
test = pd.concat([test, test_numerical_df], axis=1)